# Model initialization and definition of helpers function

In [4]:
import torch
from transformers import BertTokenizer
from models import BERTForQuantification

def load_model(checkpoint_path, device):
    """
    Loads the fine-tuned model from a saved checkpoint.
    """
    # Initialize the model architecture
    model = BERTForQuantification()

    # Load the trained weights directly
    model.load_state_dict(torch.load(checkpoint_path))

    # Set the model to evaluation mode
    model.eval()
    
    model.to(device)

    return model

def tokenize_input(text, tokenizer, max_len=128):
    tokens = tokenizer(text, padding='max_length', max_length=max_len, truncation=True, return_tensors="pt")
    return tokens['input_ids'], tokens['attention_mask']

def inference(text, class_type, model, tokenizer, device):
    input_ids, attention_mask = tokenize_input(text, tokenizer)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        score = model(input_ids, attention_mask, class_type)
    return score.item() * 100


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model('trained_models/final_bert_model.pth', device)

/home/ilias/Desktop/safe-6g/Cognitive/Code/repo/Cognitive_Coordinator/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1404417/3205139007.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_

# Safety

In [5]:
input_text = "Can you fulfil my need for basic authorization?"
safety_score1 = inference(input_text, "Safety", model, tokenizer, device)
print(f"Safety Score: {round(safety_score1, 2)}")


input_text = "My application has a requirement for zero-trust authorization"
safety_score2 = inference(input_text, "Safety", model, tokenizer, device)
print(f"Safety Score: {round(safety_score2, 2)}")

Safety Score: 37.59
Safety Score: 47.57


# Security

In [6]:
input_text = "I am scared of phishing attacks"
security_score1 = inference(input_text, "Security", model, tokenizer, device)
print(f"Security Score: {round(security_score1, 2)}")


input_text = "Protect me from data breaches"
security_score2 = inference(input_text, "Security", model, tokenizer, device)
print(f"Security Score: {round(security_score2, 2)}")

Security Score: 69.18
Security Score: 76.99


# Privacy

In [7]:
input_text = "I care about my login data"
privacy_score1 = inference(input_text, "Privacy", model, tokenizer, device)
print(f"Privacy Score: {round(privacy_score1, 2)}")


input_text = "I care about my login data and my browser history"
privacy_score2 = inference(input_text, "Privacy", model, tokenizer, device)
print(f"Privacy Score: {round(privacy_score2, 2)}")

Privacy Score: 30.82
Privacy Score: 40.57


# Reliability

In [9]:
input_text = "I want my application to be reliable"
reliability_score1 = inference(input_text, "Reliability", model, tokenizer, device)
print(f"Reliability Score: {round(reliability_score1, 2)}")


input_text = "My application will use DNNs (Deep Neural Networks)."
reliability_score2 = inference(input_text, "Reliability", model, tokenizer, device)
print(f"Reliability Score: {round(reliability_score2, 2)}")

Reliability Score: 45.73
Reliability Score: 60.62


# Resilience

In [10]:
input_text = "I care about connection's speed"
resilience_score1 = inference(input_text, "Resilience", model, tokenizer, device)
print(f"Resilience Score: {round(resilience_score1, 2)}")


input_text = "Also, I want to restore the latest state of my app"
resilience_score2 = inference(input_text, "Resilience", model, tokenizer, device)
print(f"Resilience Score: {round(resilience_score2, 2)}")

Resilience Score: 55.96
Resilience Score: 65.13


# Overall non-calibrated LoTw

In [11]:
safety_score = round((safety_score1+safety_score2)/2, 2)

security_score = round((security_score1+security_score2)/2, 2)

privacy_score = round((privacy_score1+privacy_score2)/2, 2)

reliability_score = round((reliability_score1+reliability_score2)/2, 2)

resilience_score = round((resilience_score1+resilience_score2)/2, 2)

scores = [safety_score, security_score, privacy_score, reliability_score, resilience_score]

print(f"Safety score: {safety_score}")
print(f"Security score: {security_score}")
print(f"Privacy score: {privacy_score}")
print(f"Reliability score: {reliability_score}")
print(f"Resilience score: {reliability_score}")
print(f"First Trustworthiness score: {round((safety_score+security_score+privacy_score+reliability_score+resilience_score)/5,2)}")

Safety score: 42.58
Security score: 73.08
Privacy score: 35.7
Reliability score: 53.18
Resilience score: 53.18
First Trustworthiness score: 53.02


# Callibration

## Get all resources

In [12]:
from callibrate import *

config_file = "/home/ilias/Desktop/safe-6g/Cognitive/Code/repo/Cognitive_Coordinator/TF_Configuration_Files/reliability.yaml"
api_url = "https://safe-6g-ncsrd.satrd.es/entities?type=InfrastructureElement&format=simplified"
token = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJzaTcxSzNkUm11UFIxY2RhT2daNVFtbGpUVlR6U3JQM0cyYlZNdEVDeUVjIn0.eyJleHAiOjE4MDk0MjgwMDMsImlhdCI6MTcyMzExNDQwMywianRpIjoiMzI4OTdiMWEtNjFjMy00Yzk0LTkwN2QtZDg0Y2Y1NTQwOTNhIiwiaXNzIjoiaHR0cHM6Ly9rZXljbG9hay5jZi1tdnAtZG9tYWluLmFlcm9zLXByb2plY3QuZXUvYXV0aC9yZWFsbXMva2V5Y2xvYWNrLW9wZW5sZGFwIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjU5MzMyMDg2LTA0YzgtNGFiZS1iY2JiLWEyZjMzMmM3M2FmYSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFlcm9zLXRlc3QiLCJzZXNzaW9uX3N0YXRlIjoiN2YyOWMyMzEtMWY2Zi00YzNkLThhMDEtMjVmZjVmMjMwNDA2IiwiYWNyIjoiMSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJDbG91ZEZlcnJvRG9tYWluIiwiZGVmYXVsdC1yb2xlcy1rZXljbG9hY2stb3BlbmxkYXAiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiRG9tYWluIGFkbWluaXN0cmF0b3IiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6InByb2ZpbGUgZW1haWwiLCJzaWQiOiI3ZjI5YzIzMS0xZjZmLTRjM2QtOGEwMS0yNWZmNWYyMzA0MDYiLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsIm5hbWUiOiJEb21haW4gYWRtaW5pc3RyYXRvciAxIEFkbWluIiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZG9tYWluYWRtaW5pc3RyYXRvcjEiLCJnaXZlbl9uYW1lIjoiRG9tYWluIGFkbWluaXN0cmF0b3IgMSIsImZhbWlseV9uYW1lIjoiQWRtaW4ifQ.dF3CS5Wq23YB2sbc2-epH4QLhN9Y9JGuqW7aQ5x0mGgM4v_bkmycXnceuKohVxgSreSq5jJ-m7P38-HGfX0GoLDiSENqlw8SzMyKmjlFuu5rreXIbskI3GKqbfGog4ZR8ojTCCfbfwgdZsvc_XFZRnrsC_nFuHe2AiD3ypWPFnEY9edvzG-oWC414hvIHGLdVAXqLthWJe65s1QfOWrn70lvBuszHDg48iec_zv0Us5u8yeYXahO8Tf7FrQ4CgGuocS2vn55ENQgLDs03E01m6CWPlANhgJKEfziPGCxRuYKIDNZOrvhIF-ZsMEsrt95jg-qeskqkdA2dWzbJwM5Sw"

# Load configuration
#config = load_config(config_file)
prolog = load_prolog_knowledge("TF_Configuration_Files/reliability.pl")

# Fetch available resources
resources = get_available_resources(api_url, token)

# Print all available resources
for resource in resources:
    print(f"Hostname: {resource['hostname']}, CPU: {resource['available_cpu']:.2f}, Memory: {resource['available_memory']}")



Hostname: ncsrd-worker3, CPU: 1.80, Memory: 11.76Gi
Hostname: ncsrd-worker, CPU: 2.85, Memory: 11.86Gi
Hostname: ubuntu, CPU: 2.04, Memory: 10.34Gi
Hostname: duzunidis-vm1, CPU: 7.68, Memory: 7.03Gi
Hostname: duzunidis-vm3, CPU: 7.60, Memory: 9.71Gi
Hostname: duzunidis-vm2, CPU: 7.84, Memory: 8.93Gi
Hostname: cloud, CPU: 5.52, Memory: 25.06Gi
Hostname: worker2, CPU: 3.80, Memory: 13.15Gi
Hostname: worker1, CPU: 3.92, Memory: 6.27Gi
Hostname: worker3, CPU: 3.96, Memory: 6.63Gi


## Get best resource option

In [17]:
# Sort resources by available CPU and memory in descending order
resources_sorted = sorted(
    resources,
    key=lambda r: (r['available_cpu'], float(r['available_memory'][:-2])),  # Convert memory to float for sorting
    reverse=True
)

# Select the infrastructure element with the most free resources
best_resource = resources_sorted[0]
available_cpu = best_resource['available_cpu']
available_memory = best_resource['available_memory']
#available_memory = '600'
print(f"\nUsing best resource: Hostname: {best_resource['hostname']}, CPU: {available_cpu:.2f}, Memory: {available_memory}")



Using best resource: Hostname: duzunidis-vm2, CPU: 7.84, Memory: 8.93Gi


## Callibrate the scores

In [18]:
# Convert available_memory to MB once
if available_memory.endswith('Gi'):
    available_memory = float(available_memory.replace('Gi', '')) * 1024
elif available_memory.endswith('Mi'):
    available_memory = float(available_memory.replace('Mi', ''))

# Process scores
for score in scores:
    flavor = get_flavor(prolog, score, available_cpu, available_memory)
    print(f"Score: {score}, Assigned Flavor: {flavor}")

Score: 42.58, Assigned Flavor: medium_reliability
Score: 73.08, Assigned Flavor: medium_reliability
Score: 35.7, Assigned Flavor: low_reliability
Score: 53.18, Assigned Flavor: medium_reliability
Score: 60.55, Assigned Flavor: medium_reliability
